# Mode d'emploi

Ce script permet de changer le SCR d'un ou plusieurs shapefiles.

0. Créer sur le serveur un répertoire input/ et un répertoire output/

1. Les fichiers correspondant au(x) shapefile(s) d'entrée devront être placés dans une archive ZIP avant d'être uploadés sur le serveur Jupyter. Les fichiers shapefiles à convertir doivent toujours se trouver au 2e niveau de l'arborescence de l'archive :
  * dir1/
    * dir1/foo1.shp,
    * dir1/foo1.prj,
    * ...
    * dir1/bar1.shp,
    * ...
  * dir2/
    * ...

**Exemple 1** : dans le cas d'un unique shapefile représentant les départements métropolitains français, on placera ces fichiers (.shp, .prj, .dbf, etc.) dans un répertoire departements_metropole/ et on compressera ce répertoire. Ainsi, lors de l'ouverture de l'archive ZIP avec un explorateur, on doit voir un répertoire racine puis, une fois à l'intérieur, les fichiers du shapefile.

**Exemple 2** : dans le cas de shapefiles séparés représentant les routes de chaque département, ces fichiers étant rangés dans des répertoires routes/01/, routes/02/, etc., on sélectionnera l'ensemble des répertoires 01/, 02/, etc. pour les ajouter à l'archive. Ainsi, lors de l'ouverture de l'archive ZIP avec un explorateur, on doit voir à la racine 01/, 02/, etc., chacun contenant les fichiers .shp, .prj, .dbf, etc.

2. Uploader l'archive ZIP dans le répertoire input/
3. Indiquer le nom de l'archive et le code EPSG de destination
4. Exécuter les cellules correspondant à l'outil de reprojection ci-après
5. Télécharger l'archive ZIP depuis le répertoire output/

**Remarque** : les répertoires du premier niveau de l'arborescence du ZIP sont décompressés un à un ; il est donc nécessaire de disposer d'un espace équivalent à deux fois la taille compressée de l'archive (entrée + sortie) plus deux fois la taille décompressée d'un répertoire du premier niveau (shapefiles originaux et reprojetés).

# Outil de reprojection

In [ ]:
'''
Conversion de shapefiles vers un SCR spécifié (d'après son EPSG)
'''
import os
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
# Répertoire et nom de l'archive contenant les shapefiles d'entrée
input_dir = "input"
input_zip = "IGN_routes"

input_zip_fpath = input_dir + "/" + input_zip + ".zip"

# EPSG de sortie
# Lambert zone II = 7421 ou 27572
# Lambert 93 = 2154
epsg_out = 7421

# Répertoire devant accueillir les shapefiles de sortie
output_dir = "output"
output_zip_fpath = "../../"+ output_dir + "/" + input_zip + "_EPSG" + str(epsg_out) + ".zip"

In [ ]:
# Liste des répertoires contenus dans le zip
list_dir_names = !unzip -Z1 $input_zip_fpath "*/"

print("Contenu de l'archive :")
for dir_name in list_dir_names:
    print(dir_name)
    

In [ ]:
os.makedirs("temp/in")
os.makedirs("temp/out")

for dir_name in list_dir_names:
    print("Traitement des shapefiles du répertoire : " + dir_name)
    
    # décompression du répertoire de l'archive dans temp/in
    dir_path = "temp/in/" + dir_name
    to_unzip = dir_name + "*"
    !unzip -q $input_zip_fpath $to_unzip -d "temp/in/"
    
    # création du répertoire temporaire de sortie
    shp_reprojete_dir = "temp/out/" + dir_name
    os.makedirs("temp/out/" + dir_name)
    
    # parcours des répertoires et réécriture
    for input_file in os.listdir("temp/in/" + dir_name):
        if input_file.endswith(".shp") | input_file.endswith(".SHP"):
            # lecture du shapefile
            input_shp = gpd.read_file("temp/in/" + dir_name + input_file)
            print("SCR du shapefile " + input_file + " : ", input_shp.crs)
            
            # conversion
            output_shp = input_shp.to_crs(epsg=epsg_out)
            
            # ecriture du nouveau shapefile
            output_shp.to_file("temp/out/" + dir_name + "/" + input_file)
            
    # recompression des fichiers de sortie dans l'archive résultat
    # N.B. qu'il est nécessaire de se déplacer sinon l'archive contient l'arborescence temp/out/etc.
    %cd -q temp/out/
    print("Compression de " + shp_reprojete_dir)
    !zip -rq $output_zip_fpath $dir_name
    %cd -q ../../
    
    # suppression des fichiers d'entrée et de sortie
    !rm -r $dir_path
    !rm -r $shp_reprojete_dir

# Supprimer les fichiers temporaires
!rm -r "temp/in"
!rm -r "temp/out"

# Annexe : fonctions utiles

In [ ]:
# Supprimer les fichiers temporaires
!rm -r "temp/in"
!rm -r "temp/out"

In [ ]:
# Compression d'un répertoire
src = "input/dossier_a_archiver"
archive_cbl = "output/archive.zip"

!zip -r $archive_cbl $src

In [ ]:
# Décompression d'un répertoire et de tout ce qu'il contient
archive_src = "input/archive.zip"
cbl = "output/dossier_decompresse"

!unzip $archive_src -d $cbl